## Functions

### Never Unpack More Than Three Variables When Functions Return Multiple Values

python支持函数多返回值，支持unpacking。

导致我也偶尔写过这样的代码(

`a, b, c, d, e, f, g, h, i = func()`

这样的代码有两个明显的问题

- 顺序很容易出错，需要对照检查，出错还很难发现。。
- 可读性较差（如果代码要求pylint检查的话，行长度限制会使得这个语句七零八落）

因此

- 返回值不要超过3个
- 否则， return a small class or namedtuple instance来替代
> transformers中就使用了这种写法


### Prefer Raising Exceptions to Returning None

return None导致问题的最常见情景是：

主过程中用`if not result`，而不是`if result is not None`来判断。而函数有正确的返回值0, 或是空字符串。

一种常见的解决方式是返回一个tuple, `(flag, result)`，但调用者很容易就会忽视掉flag, 写起来也显得冗余

因此推荐的写法就是在函数中raise, 由调用者来handle.
- 复杂的API推荐派生Exception, 实现自己的异常处理类。

In [ ]:
# 一个完整的例子，包括type annotation和docstring
def careful_divide(a: float, b: float) -> float:
"""Divides a by b.

Raises:
ValueError: When the inputs cannot be divided.
"""
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')